## 基本流程

&emsp;&emsp;概念：基于树模型做决策；**每个节点对应某个属性**；每个分支对应可能的结果；叶子结点对应预测结果。

&emsp;&emsp;每个节点对应的判断属性只是一个性质，就是我们依据什么分叉。

## 划分选择

### 信息增益(ID3)


#### 信息熵的概念与度量

##### 概念

1. **熵**

&emsp;&emsp;一种事物的不确定性叫做熵。

2. **信息**

&emsp;&emsp;信息说的是一种能够消除不确定性的事物，能够用来调整概率、排除干扰。

&emsp;&emsp;**确定情况**是：比如卖瓜的人说了一句，保熟保甜，开了不甜算我的。

3. **噪音**

&emsp;&emsp;不能消除某人对某件事物的不确定性。

4. **数据**

&emsp;&emsp;我们所能拿到的数据就是噪音加上信息。

##### 熵如何量化

&emsp;&emsp;以抛硬币为例，假设不确定性有`8`种，那么抛硬币就需要抛$log_{2}^{8} = 3$种。取完对数之后就能够去衡量这个熵了。

&emsp;&emsp;如果是每种情况概率不想等的话，一般分布可以描述为:

$$
\operatorname{Ent}(D)=-\sum_{k=1}^{|\mathcal{Y}|} p_{k} \log _{2} p_{k}
$$

#### 信息如何量化

&emsp;&emsp;当知道了熵如何量化之后，我们就能够去知道说知道了多少信息。

&emsp;&emsp;在得知信息的前后，我的不确定性的变化。也就是**对这个信息不确定性的差额就是信息提供的信息量**。



&emsp;&emsp;拿到一个数据集之后，根节点的信息增益，就是依据标签计算出来的。

&emsp;&emsp;之后再计算每个特征划分完之后，里面有多少正例、里面有多少反例。比如依据色泽划分完之后，里面有**青绿**、**乌黑**和**浅白**。

1. 青绿的有六分之三的正样本，六分之三的负样本；

2. 乌黑的有六分之四的正样本，六分之二的负样本；

3. 浅白的里面有五分之一的正样本，五分之四的负样本。

&emsp;&emsp;然后计算熵减了多少即可，减少最多的就说明信息越多。

&emsp;&emsp;**熵减最大**：

$$
\Delta_{Entropy Reduction} = Entropy(parent) - \sum_{j=1}^{k} \frac{N(j)}{N}Entropy(j)
$$

&emsp;&emsp;$k$是划分属性取值的个数。$N$是父亲结点上样本的总数，$N(j)$是第$j$个儿子结点上样本的数目。



### 增益率(C4.5)

&emsp;&emsp;如果以信息增益率为划分依据，存在偏向选择**特征取值较多的特征**，信息增益比是对这一问题进行矫正。比如划分用户的时候，依据用户`ID`划分的话，只需要一层决策树就够了。

- 实际上，信息增益准则对可取数目较多的属性有所偏好，`C4.5`决策树算法不直接使用信息增益，而是使用**增益率**(`gain ratio`)来选择最优划分属性。增益率定义为:

$$
\operatorname{Gain} \operatorname{ratio}(D, a)=\frac{\operatorname{Gain}(D, a)}{\operatorname{IV}(a)}
$$

&emsp;&emsp;其中:

$$
\operatorname{IV}(a)=-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \log _{2} \frac{\left|D^{v}\right|}{|D|}
$$

&emsp;&emsp;属性$a$的可能取值数目越多(即V越大)，则$IV(a)$的取值通常会越大。

&emsp;&emsp;在具体使用的时候，先从候选划分属性中找出信息增益高于平均水平的属性，再从中选择增益率最高的。

### 基尼指数(CART)

&emsp;&emsp;我们也可以将纯度的计算换成基尼指数：

$$
Gini(t)=1-\sum_{i=1}^{c}p(i)^{2}
$$

&emsp;&emsp;假设我们依据`Chest Pain`特征划分，之后得到是否患心脏病的统计数字为: `yes-105`，`no-39`；`yes-34`，`no-125`。

In [1]:
def gini_index_single(a, b):
    single_gini = 1 - ((a / (a + b))**2) - ((b / (a + b))**2)
    return round(single_gini, 2)

def gini_index(a, b, c, d):
    left = gini_index_single(a, b)
    right = gini_index_single(c, d)
    
    # 算出左边和右边之后再进行加权平均。
    gini_index = left * ((a + b) / (a + b + c + d)) + right * ((c + d) / (a + b + c + d))
    return round(gini_index, 2)

print(gini_index_single(105, 39))
print(gini_index(105, 39, 34, 125))

0.39
0.36


#### 分类树

&emsp;&emsp;基尼指数最小准则。与熵减最小类似。

#### 回归树

&emsp;&emsp;平方误差最小准则。

## 剪枝处理

&emsp;&emsp;剪枝的目的就是为了对付过拟合。

### 预剪枝

&emsp;&emsp;选择一个特征，在不分的时候，查看一下验证集上的精度(依据训练集上的正负样本的比例来判断)。如果划分之后，精度有提升的话，就划分，不然就不划分。

### 后剪枝

&emsp;&emsp;就是全部分完，之后再看能不剪枝。如果剪枝前后没有变化，就不剪了。预剪枝是能剪的全部剪掉。

## 连续与缺失值

### 连续值处理

&emsp;&emsp;使用二分法，找到能够使得信息增益减最大的阈值。

### 缺失值处理

&emsp;&emsp;属性里面有一些缺失值如何处理？

1. **C4.5例子**: 去掉有缺失值的样本，计算一遍信息增益。然后再乘上权重(去掉有缺失值的样本 / 全部样本)。

2. **离散值**：
    - 众数填充；
    - 相关性最高的列填充；

3. **连续值**：
    - 中位数；
    - 相关性最高的列做线性回归进行估计；

## 多变量决策树

&emsp;&emsp;多变量决策树里面判断节点分叉的地方可能是多个属性的组合。里面存在属性的线性组合，分割曲线会比较平滑。

## 参考资料